In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns
import os

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [4]:
D = 30
N = 10000
z_size = 8

# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.0001
b1 = 0.9
b2 = 0.999

global_t = 4
k_lab = [D//10, D//6, D//3, D//2, D]
trial_num = 1

In [5]:
train_data, test_data = generate_synthetic_data_with_noise(N, z_size, D)

In [6]:
def train_model(train_data, model):
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=lr, 
                                 betas = (b1,b2))
        
    for epoch in range(1, n_epochs+1):
        train(train_data, 
              model, 
              optimizer, 
              epoch, 
              batch_size)
        model.t = max(0.001, model.t * 0.99)

        
    return model

In [7]:
def save_model(base_path, model):
    # make directory
    if not os.path.exists(os.path.dirname(base_path)):
        try:
            os.makedirs(os.path.dirname(base_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise Exception("COULD NOT MAKE PATH")
    with open(base_path, 'wb') as PATH:
        torch.save(model.state_dict(), PATH)

In [8]:
BASE_PATH = "../data/models/final_run/"
#BASE_PATH = '/scratch/ns3429/sparse-subset/data/models/final_run/'

Vanilla VAE Gumbel First

In [9]:
def run_vanilla_vae_gumbel():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("VANILLA VAE GUMBEL, TRIAL={}, K={}".format(trial, k))
            additional_path = "vanilla_vae_gumbel/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [10]:
#run_vanilla_vae_gumbel()

Batching Gumbel VAE

In [11]:
def run_batching_gumbel_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("BATCHING GUMBEL VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "batching_gumbel_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_NInsta(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [12]:
#run_batching_gumbel_vae()

Global Gate VAE

In [13]:
def run_globalgate_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("GLOBAL GATE VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "globalgate_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_GlobalGate(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [14]:
#run_globalgate_vae()

RunningState VAE

In [24]:
def run_runningstate_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("RUNNING STATE VAE, TRIAL={}, K={}".format(trial, k))
            additional_path = "runningstate_vae/k_{}/model_trial_{}.pt".format(k, trial)
            model = VAE_Gumbel_RunningState(2*D, 100, 20, k = k, t = global_t, alpha = 0.9)
            model.to(device)
            train_model(train_data, model)
            model.set_burned_in()
            save_model(BASE_PATH + additional_path, model)

In [25]:
#run_runningstate_vae()

ConcreteVAE

In [17]:
def run_concrete_vae():
    for trial in range(1, trial_num+1):
        for k in k_lab:
            print("CONCRETE VAE NMSL, TRIAL={}, K={}".format(trial, k))
            additional_path = "concrete_vae_nmsl/k_{}/model_trial_{}.pt".format(k, trial)
            model = ConcreteVAE_NMSL(2*D, 100, 20, k = k, t = global_t)
            model.to(device)
            train_model(train_data, model)
            save_model(BASE_PATH + additional_path, model)

In [18]:
#run_concrete_vae()

CONCRETE VAE NMSL, TRIAL=1, K=3
Train Epoch: 1 [0/8000 (0%)]	Loss: 42.180851
Train Epoch: 1 [1280/8000 (16%)]	Loss: 41.562439
Train Epoch: 1 [2560/8000 (32%)]	Loss: 41.008808
Train Epoch: 1 [3840/8000 (48%)]	Loss: 40.438026
Train Epoch: 1 [5120/8000 (64%)]	Loss: 39.695099
Train Epoch: 1 [6400/8000 (80%)]	Loss: 39.106960
Train Epoch: 1 [7680/8000 (96%)]	Loss: 38.528366
====> Epoch: 1 Average loss: 40.3232
Train Epoch: 2 [0/8000 (0%)]	Loss: 38.329620
Train Epoch: 2 [1280/8000 (16%)]	Loss: 37.836788
Train Epoch: 2 [2560/8000 (32%)]	Loss: 36.907288
Train Epoch: 2 [3840/8000 (48%)]	Loss: 36.531895
Train Epoch: 2 [5120/8000 (64%)]	Loss: 35.862644
Train Epoch: 2 [6400/8000 (80%)]	Loss: 35.276237
Train Epoch: 2 [7680/8000 (96%)]	Loss: 34.927074
====> Epoch: 2 Average loss: 36.4796
Train Epoch: 3 [0/8000 (0%)]	Loss: 34.901760
Train Epoch: 3 [1280/8000 (16%)]	Loss: 34.385445
Train Epoch: 3 [2560/8000 (32%)]	Loss: 34.056343
Train Epoch: 3 [3840/8000 (48%)]	Loss: 34.323883
Train Epoch: 3 [5120/800

Train Epoch: 22 [6400/8000 (80%)]	Loss: 32.598652
Train Epoch: 22 [7680/8000 (96%)]	Loss: 32.206890
====> Epoch: 22 Average loss: 32.3652
Train Epoch: 23 [0/8000 (0%)]	Loss: 32.178741
Train Epoch: 23 [1280/8000 (16%)]	Loss: 32.235870
Train Epoch: 23 [2560/8000 (32%)]	Loss: 32.431625
Train Epoch: 23 [3840/8000 (48%)]	Loss: 32.618336
Train Epoch: 23 [5120/8000 (64%)]	Loss: 32.183876
Train Epoch: 23 [6400/8000 (80%)]	Loss: 32.248032
Train Epoch: 23 [7680/8000 (96%)]	Loss: 32.176655
====> Epoch: 23 Average loss: 32.3415
Train Epoch: 24 [0/8000 (0%)]	Loss: 32.335918
Train Epoch: 24 [1280/8000 (16%)]	Loss: 32.268177
Train Epoch: 24 [2560/8000 (32%)]	Loss: 32.491188
Train Epoch: 24 [3840/8000 (48%)]	Loss: 32.574436
Train Epoch: 24 [5120/8000 (64%)]	Loss: 32.931740
Train Epoch: 24 [6400/8000 (80%)]	Loss: 32.578922
Train Epoch: 24 [7680/8000 (96%)]	Loss: 32.372932
====> Epoch: 24 Average loss: 32.2986
Train Epoch: 25 [0/8000 (0%)]	Loss: 31.979761
Train Epoch: 25 [1280/8000 (16%)]	Loss: 32.24314

Train Epoch: 44 [1280/8000 (16%)]	Loss: 32.436314
Train Epoch: 44 [2560/8000 (32%)]	Loss: 32.624523
Train Epoch: 44 [3840/8000 (48%)]	Loss: 32.017361
Train Epoch: 44 [5120/8000 (64%)]	Loss: 32.025887
Train Epoch: 44 [6400/8000 (80%)]	Loss: 32.187412
Train Epoch: 44 [7680/8000 (96%)]	Loss: 32.172543
====> Epoch: 44 Average loss: 32.1568
Train Epoch: 45 [0/8000 (0%)]	Loss: 32.258099
Train Epoch: 45 [1280/8000 (16%)]	Loss: 32.354954
Train Epoch: 45 [2560/8000 (32%)]	Loss: 32.427200
Train Epoch: 45 [3840/8000 (48%)]	Loss: 32.312920
Train Epoch: 45 [5120/8000 (64%)]	Loss: 32.274895
Train Epoch: 45 [6400/8000 (80%)]	Loss: 32.166805
Train Epoch: 45 [7680/8000 (96%)]	Loss: 31.682611
====> Epoch: 45 Average loss: 32.1593
Train Epoch: 46 [0/8000 (0%)]	Loss: 32.381916
Train Epoch: 46 [1280/8000 (16%)]	Loss: 32.331970
Train Epoch: 46 [2560/8000 (32%)]	Loss: 31.684504
Train Epoch: 46 [3840/8000 (48%)]	Loss: 32.122700
Train Epoch: 46 [5120/8000 (64%)]	Loss: 32.122570
Train Epoch: 46 [6400/8000 (80%)

Train Epoch: 65 [5120/8000 (64%)]	Loss: 31.804993
Train Epoch: 65 [6400/8000 (80%)]	Loss: 32.339897
Train Epoch: 65 [7680/8000 (96%)]	Loss: 32.212585
====> Epoch: 65 Average loss: 32.0908
Train Epoch: 66 [0/8000 (0%)]	Loss: 31.744354
Train Epoch: 66 [1280/8000 (16%)]	Loss: 32.306541
Train Epoch: 66 [2560/8000 (32%)]	Loss: 32.524021
Train Epoch: 66 [3840/8000 (48%)]	Loss: 31.983976
Train Epoch: 66 [5120/8000 (64%)]	Loss: 32.128967
Train Epoch: 66 [6400/8000 (80%)]	Loss: 32.307137
Train Epoch: 66 [7680/8000 (96%)]	Loss: 32.185196
====> Epoch: 66 Average loss: 32.0882
Train Epoch: 67 [0/8000 (0%)]	Loss: 31.879133
Train Epoch: 67 [1280/8000 (16%)]	Loss: 31.937485
Train Epoch: 67 [2560/8000 (32%)]	Loss: 32.407833
Train Epoch: 67 [3840/8000 (48%)]	Loss: 31.912764
Train Epoch: 67 [5120/8000 (64%)]	Loss: 31.837276
Train Epoch: 67 [6400/8000 (80%)]	Loss: 32.105637
Train Epoch: 67 [7680/8000 (96%)]	Loss: 32.236988
====> Epoch: 67 Average loss: 32.0848
Train Epoch: 68 [0/8000 (0%)]	Loss: 32.10792

Train Epoch: 87 [2560/8000 (32%)]	Loss: 31.921621
Train Epoch: 87 [3840/8000 (48%)]	Loss: 32.013695
Train Epoch: 87 [5120/8000 (64%)]	Loss: 31.617136
Train Epoch: 87 [6400/8000 (80%)]	Loss: 31.955040
Train Epoch: 87 [7680/8000 (96%)]	Loss: 31.732883
====> Epoch: 87 Average loss: 31.9427
Train Epoch: 88 [0/8000 (0%)]	Loss: 32.460163
Train Epoch: 88 [1280/8000 (16%)]	Loss: 31.893515
Train Epoch: 88 [2560/8000 (32%)]	Loss: 31.893162
Train Epoch: 88 [3840/8000 (48%)]	Loss: 31.982914
Train Epoch: 88 [5120/8000 (64%)]	Loss: 31.992805
Train Epoch: 88 [6400/8000 (80%)]	Loss: 31.491026
Train Epoch: 88 [7680/8000 (96%)]	Loss: 32.287613
====> Epoch: 88 Average loss: 31.9486
Train Epoch: 89 [0/8000 (0%)]	Loss: 32.156601
Train Epoch: 89 [1280/8000 (16%)]	Loss: 32.236046
Train Epoch: 89 [2560/8000 (32%)]	Loss: 31.943233
Train Epoch: 89 [3840/8000 (48%)]	Loss: 31.817421
Train Epoch: 89 [5120/8000 (64%)]	Loss: 31.701342
Train Epoch: 89 [6400/8000 (80%)]	Loss: 32.452095
Train Epoch: 89 [7680/8000 (96%)

...

Train Epoch: 515 [5120/8000 (64%)]	Loss: 30.907618
Train Epoch: 515 [6400/8000 (80%)]	Loss: 29.976423
Train Epoch: 515 [7680/8000 (96%)]	Loss: 30.639624
====> Epoch: 515 Average loss: 30.3819
Train Epoch: 516 [0/8000 (0%)]	Loss: 30.116720
Train Epoch: 516 [1280/8000 (16%)]	Loss: 30.262320
Train Epoch: 516 [2560/8000 (32%)]	Loss: 29.866848
Train Epoch: 516 [3840/8000 (48%)]	Loss: 30.714266
Train Epoch: 516 [5120/8000 (64%)]	Loss: 30.198717
Train Epoch: 516 [6400/8000 (80%)]	Loss: 30.489468
Train Epoch: 516 [7680/8000 (96%)]	Loss: 29.847935
====> Epoch: 516 Average loss: 30.3894
Train Epoch: 517 [0/8000 (0%)]	Loss: 30.567596
Train Epoch: 517 [1280/8000 (16%)]	Loss: 30.405926
Train Epoch: 517 [2560/8000 (32%)]	Loss: 30.485270
Train Epoch: 517 [3840/8000 (48%)]	Loss: 30.420408
Train Epoch: 517 [5120/8000 (64%)]	Loss: 30.300512
Train Epoch: 517 [6400/8000 (80%)]	Loss: 30.359428
Train Epoch: 517 [7680/8000 (96%)]	Loss: 30.184114
====> Epoch: 517 Average loss: 30.3791
Train Epoch: 518 [0/8000

Train Epoch: 537 [1280/8000 (16%)]	Loss: 30.003546
Train Epoch: 537 [2560/8000 (32%)]	Loss: 30.377544
Train Epoch: 537 [3840/8000 (48%)]	Loss: 30.318302
Train Epoch: 537 [5120/8000 (64%)]	Loss: 30.460144
Train Epoch: 537 [6400/8000 (80%)]	Loss: 30.392103
Train Epoch: 537 [7680/8000 (96%)]	Loss: 30.148716
====> Epoch: 537 Average loss: 30.3551
Train Epoch: 538 [0/8000 (0%)]	Loss: 30.698635
Train Epoch: 538 [1280/8000 (16%)]	Loss: 30.350828
Train Epoch: 538 [2560/8000 (32%)]	Loss: 30.391439
Train Epoch: 538 [3840/8000 (48%)]	Loss: 30.283321
Train Epoch: 538 [5120/8000 (64%)]	Loss: 30.029919
Train Epoch: 538 [6400/8000 (80%)]	Loss: 30.514202
Train Epoch: 538 [7680/8000 (96%)]	Loss: 30.624874
====> Epoch: 538 Average loss: 30.3645
Train Epoch: 539 [0/8000 (0%)]	Loss: 30.681826
Train Epoch: 539 [1280/8000 (16%)]	Loss: 30.111393
Train Epoch: 539 [2560/8000 (32%)]	Loss: 30.341301
Train Epoch: 539 [3840/8000 (48%)]	Loss: 30.344738
Train Epoch: 539 [5120/8000 (64%)]	Loss: 30.331533
Train Epoch:

Train Epoch: 558 [2560/8000 (32%)]	Loss: 30.560917
Train Epoch: 558 [3840/8000 (48%)]	Loss: 30.204929
Train Epoch: 558 [5120/8000 (64%)]	Loss: 30.117163
Train Epoch: 558 [6400/8000 (80%)]	Loss: 30.177631
Train Epoch: 558 [7680/8000 (96%)]	Loss: 30.265259
====> Epoch: 558 Average loss: 30.3555
Train Epoch: 559 [0/8000 (0%)]	Loss: 30.249487
Train Epoch: 559 [1280/8000 (16%)]	Loss: 30.317320
Train Epoch: 559 [2560/8000 (32%)]	Loss: 29.998362
Train Epoch: 559 [3840/8000 (48%)]	Loss: 30.321543
Train Epoch: 559 [5120/8000 (64%)]	Loss: 30.219658
Train Epoch: 559 [6400/8000 (80%)]	Loss: 30.126614
Train Epoch: 559 [7680/8000 (96%)]	Loss: 30.561485
====> Epoch: 559 Average loss: 30.3290
Train Epoch: 560 [0/8000 (0%)]	Loss: 30.408583
Train Epoch: 560 [1280/8000 (16%)]	Loss: 30.095734
Train Epoch: 560 [2560/8000 (32%)]	Loss: 30.234348
Train Epoch: 560 [3840/8000 (48%)]	Loss: 30.288727
Train Epoch: 560 [5120/8000 (64%)]	Loss: 30.623774
Train Epoch: 560 [6400/8000 (80%)]	Loss: 30.526924
Train Epoch:

Train Epoch: 579 [3840/8000 (48%)]	Loss: 30.490437
Train Epoch: 579 [5120/8000 (64%)]	Loss: 30.410925
Train Epoch: 579 [6400/8000 (80%)]	Loss: 30.105513
Train Epoch: 579 [7680/8000 (96%)]	Loss: 30.534460
====> Epoch: 579 Average loss: 30.3426
Train Epoch: 580 [0/8000 (0%)]	Loss: 30.807974
Train Epoch: 580 [1280/8000 (16%)]	Loss: 30.399656
Train Epoch: 580 [2560/8000 (32%)]	Loss: 30.294176
Train Epoch: 580 [3840/8000 (48%)]	Loss: 30.132492
Train Epoch: 580 [5120/8000 (64%)]	Loss: 30.526760
Train Epoch: 580 [6400/8000 (80%)]	Loss: 30.252031
Train Epoch: 580 [7680/8000 (96%)]	Loss: 30.599310
====> Epoch: 580 Average loss: 30.3467
Train Epoch: 581 [0/8000 (0%)]	Loss: 30.391983
Train Epoch: 581 [1280/8000 (16%)]	Loss: 30.577061
Train Epoch: 581 [2560/8000 (32%)]	Loss: 31.097403
Train Epoch: 581 [3840/8000 (48%)]	Loss: 30.423729
Train Epoch: 581 [5120/8000 (64%)]	Loss: 29.987274
Train Epoch: 581 [6400/8000 (80%)]	Loss: 30.278603
Train Epoch: 581 [7680/8000 (96%)]	Loss: 30.332281
====> Epoch:

Train Epoch: 600 [6400/8000 (80%)]	Loss: 30.256571
Train Epoch: 600 [7680/8000 (96%)]	Loss: 30.281647
====> Epoch: 600 Average loss: 30.3015


In [19]:
model = ConcreteVAE_NMSL(2*D, 100, 20, k = 15, t = global_t)
model.to(device)

ConcreteVAE_NMSL(
  (encoder): Sequential(
    (0): Linear(in_features=15, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=100, out_features=20, bias=True)
  (enc_logvar): Linear(in_features=100, out_features=20, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=60, bias=True)
    (3): Sigmoid()
  )
)

In [20]:
model.encoder[0].weight

Parameter containing:
tensor([[ 0.0291, -0.0790,  0.2433,  ...,  0.2018, -0.0262,  0.0859],
        [ 0.0115, -0.1965,  0.1929,  ..., -0.1697,  0.0472, -0.0035],
        [-0.0486,  0.1723, -0.1632,  ...,  0.1341,  0.1805, -0.1379],
        ...,
        [-0.0725,  0.0305,  0.2064,  ..., -0.0723, -0.2168,  0.2135],
        [ 0.1760,  0.0709,  0.1004,  ...,  0.0413,  0.0738, -0.2048],
        [-0.1478, -0.0439,  0.0153,  ...,  0.1322, -0.2103, -0.1857]],
       device='cuda:0', requires_grad=True)

In [21]:
model.load_state_dict(torch.load(BASE_PATH+"concrete_vae_nmsl/k_15/model_trial_1.pt"))
model.eval()

ConcreteVAE_NMSL(
  (encoder): Sequential(
    (0): Linear(in_features=15, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=100, out_features=20, bias=True)
  (enc_logvar): Linear(in_features=100, out_features=20, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=60, bias=True)
    (3): Sigmoid()
  )
)

In [22]:
print("Ran successfully")

Ran successfully


In [ ]:
run_runningstate_vae()

RUNNING STATE VAE, TRIAL=1, K=3
Train Epoch: 1 [0/8000 (0%)]	Loss: 41.920422
Train Epoch: 1 [1280/8000 (16%)]	Loss: 41.330753
Train Epoch: 1 [2560/8000 (32%)]	Loss: 40.823719
Train Epoch: 1 [3840/8000 (48%)]	Loss: 40.312798
Train Epoch: 1 [5120/8000 (64%)]	Loss: 39.928181
Train Epoch: 1 [6400/8000 (80%)]	Loss: 39.515682
Train Epoch: 1 [7680/8000 (96%)]	Loss: 38.855728
====> Epoch: 1 Average loss: 40.2862
Train Epoch: 2 [0/8000 (0%)]	Loss: 38.818031
Train Epoch: 2 [1280/8000 (16%)]	Loss: 38.309685
Train Epoch: 2 [2560/8000 (32%)]	Loss: 37.723949
Train Epoch: 2 [3840/8000 (48%)]	Loss: 37.358112
Train Epoch: 2 [5120/8000 (64%)]	Loss: 37.044849
Train Epoch: 2 [6400/8000 (80%)]	Loss: 36.318321
Train Epoch: 2 [7680/8000 (96%)]	Loss: 36.007595
====> Epoch: 2 Average loss: 37.3179
Train Epoch: 3 [0/8000 (0%)]	Loss: 35.517395
Train Epoch: 3 [1280/8000 (16%)]	Loss: 35.523376
Train Epoch: 3 [2560/8000 (32%)]	Loss: 35.132584
Train Epoch: 3 [3840/8000 (48%)]	Loss: 34.218998
Train Epoch: 3 [5120/800

Train Epoch: 22 [5120/8000 (64%)]	Loss: 32.044991
Train Epoch: 22 [6400/8000 (80%)]	Loss: 32.181362
Train Epoch: 22 [7680/8000 (96%)]	Loss: 32.578949
====> Epoch: 22 Average loss: 32.1685
Train Epoch: 23 [0/8000 (0%)]	Loss: 31.806175
Train Epoch: 23 [1280/8000 (16%)]	Loss: 32.315926
Train Epoch: 23 [2560/8000 (32%)]	Loss: 32.145187
Train Epoch: 23 [3840/8000 (48%)]	Loss: 32.113178
Train Epoch: 23 [5120/8000 (64%)]	Loss: 32.093891
Train Epoch: 23 [6400/8000 (80%)]	Loss: 31.867002
Train Epoch: 23 [7680/8000 (96%)]	Loss: 31.683229
====> Epoch: 23 Average loss: 32.1278
Train Epoch: 24 [0/8000 (0%)]	Loss: 31.529196
Train Epoch: 24 [1280/8000 (16%)]	Loss: 31.874262
Train Epoch: 24 [2560/8000 (32%)]	Loss: 32.366837
Train Epoch: 24 [3840/8000 (48%)]	Loss: 31.902332
Train Epoch: 24 [5120/8000 (64%)]	Loss: 32.229347
Train Epoch: 24 [6400/8000 (80%)]	Loss: 32.255253
Train Epoch: 24 [7680/8000 (96%)]	Loss: 31.677784
====> Epoch: 24 Average loss: 32.0853
Train Epoch: 25 [0/8000 (0%)]	Loss: 31.85665

====> Epoch: 43 Average loss: 31.6495
Train Epoch: 44 [0/8000 (0%)]	Loss: 31.707481
Train Epoch: 44 [1280/8000 (16%)]	Loss: 31.753876
Train Epoch: 44 [2560/8000 (32%)]	Loss: 31.679129
Train Epoch: 44 [3840/8000 (48%)]	Loss: 31.408388
Train Epoch: 44 [5120/8000 (64%)]	Loss: 32.439133
Train Epoch: 44 [6400/8000 (80%)]	Loss: 31.721331
Train Epoch: 44 [7680/8000 (96%)]	Loss: 31.809891
====> Epoch: 44 Average loss: 31.6512
Train Epoch: 45 [0/8000 (0%)]	Loss: 31.530020
Train Epoch: 45 [1280/8000 (16%)]	Loss: 31.489828
Train Epoch: 45 [2560/8000 (32%)]	Loss: 31.930498
Train Epoch: 45 [3840/8000 (48%)]	Loss: 31.683670
Train Epoch: 45 [5120/8000 (64%)]	Loss: 31.659374
Train Epoch: 45 [6400/8000 (80%)]	Loss: 31.328989
Train Epoch: 45 [7680/8000 (96%)]	Loss: 31.554478
====> Epoch: 45 Average loss: 31.6285
Train Epoch: 46 [0/8000 (0%)]	Loss: 31.784498
Train Epoch: 46 [1280/8000 (16%)]	Loss: 31.375837
Train Epoch: 46 [2560/8000 (32%)]	Loss: 31.368912
Train Epoch: 46 [3840/8000 (48%)]	Loss: 31.74986

Train Epoch: 65 [2560/8000 (32%)]	Loss: 31.510838
Train Epoch: 65 [3840/8000 (48%)]	Loss: 31.616505
Train Epoch: 65 [5120/8000 (64%)]	Loss: 31.540457
Train Epoch: 65 [6400/8000 (80%)]	Loss: 31.333782
Train Epoch: 65 [7680/8000 (96%)]	Loss: 31.737049
====> Epoch: 65 Average loss: 31.4966
Train Epoch: 66 [0/8000 (0%)]	Loss: 31.137489
Train Epoch: 66 [1280/8000 (16%)]	Loss: 31.532118
Train Epoch: 66 [2560/8000 (32%)]	Loss: 31.395334
Train Epoch: 66 [3840/8000 (48%)]	Loss: 31.680079
Train Epoch: 66 [5120/8000 (64%)]	Loss: 31.615721
Train Epoch: 66 [6400/8000 (80%)]	Loss: 31.421240
Train Epoch: 66 [7680/8000 (96%)]	Loss: 31.167887
====> Epoch: 66 Average loss: 31.4657
Train Epoch: 67 [0/8000 (0%)]	Loss: 31.422094
Train Epoch: 67 [1280/8000 (16%)]	Loss: 31.351622
Train Epoch: 67 [2560/8000 (32%)]	Loss: 31.297222
Train Epoch: 67 [3840/8000 (48%)]	Loss: 31.859661
Train Epoch: 67 [5120/8000 (64%)]	Loss: 31.452585
Train Epoch: 67 [6400/8000 (80%)]	Loss: 31.553186
Train Epoch: 67 [7680/8000 (96%)

Train Epoch: 86 [7680/8000 (96%)]	Loss: 31.503405
====> Epoch: 86 Average loss: 31.2684
Train Epoch: 87 [0/8000 (0%)]	Loss: 31.392717
Train Epoch: 87 [1280/8000 (16%)]	Loss: 31.155489
Train Epoch: 87 [2560/8000 (32%)]	Loss: 31.111322
Train Epoch: 87 [3840/8000 (48%)]	Loss: 31.317032
Train Epoch: 87 [5120/8000 (64%)]	Loss: 31.261831
Train Epoch: 87 [6400/8000 (80%)]	Loss: 31.665468
Train Epoch: 87 [7680/8000 (96%)]	Loss: 31.458652
====> Epoch: 87 Average loss: 31.2565
Train Epoch: 88 [0/8000 (0%)]	Loss: 31.325371
Train Epoch: 88 [1280/8000 (16%)]	Loss: 31.337944
Train Epoch: 88 [2560/8000 (32%)]	Loss: 31.551380
Train Epoch: 88 [3840/8000 (48%)]	Loss: 31.312805
Train Epoch: 88 [5120/8000 (64%)]	Loss: 30.994761
Train Epoch: 88 [6400/8000 (80%)]	Loss: 31.016605
Train Epoch: 88 [7680/8000 (96%)]	Loss: 31.031321
====> Epoch: 88 Average loss: 31.2586
Train Epoch: 89 [0/8000 (0%)]	Loss: 31.435650
Train Epoch: 89 [1280/8000 (16%)]	Loss: 31.351601
Train Epoch: 89 [2560/8000 (32%)]	Loss: 31.37681

...

Train Epoch: 415 [2560/8000 (32%)]	Loss: 30.478485
Train Epoch: 415 [3840/8000 (48%)]	Loss: 30.645800
Train Epoch: 415 [5120/8000 (64%)]	Loss: 30.610434
Train Epoch: 415 [6400/8000 (80%)]	Loss: 30.459419
Train Epoch: 415 [7680/8000 (96%)]	Loss: 30.654751
====> Epoch: 415 Average loss: 30.4422
Train Epoch: 416 [0/8000 (0%)]	Loss: 30.257515
Train Epoch: 416 [1280/8000 (16%)]	Loss: 30.544956
Train Epoch: 416 [2560/8000 (32%)]	Loss: 30.549988
Train Epoch: 416 [3840/8000 (48%)]	Loss: 30.656046
Train Epoch: 416 [5120/8000 (64%)]	Loss: 30.489498
Train Epoch: 416 [6400/8000 (80%)]	Loss: 30.462248
Train Epoch: 416 [7680/8000 (96%)]	Loss: 31.039860
====> Epoch: 416 Average loss: 30.4402
Train Epoch: 417 [0/8000 (0%)]	Loss: 30.418831
Train Epoch: 417 [1280/8000 (16%)]	Loss: 30.314995
Train Epoch: 417 [2560/8000 (32%)]	Loss: 30.570124
Train Epoch: 417 [3840/8000 (48%)]	Loss: 30.782475
Train Epoch: 417 [5120/8000 (64%)]	Loss: 30.399158
Train Epoch: 417 [6400/8000 (80%)]	Loss: 30.443567
Train Epoch:

Train Epoch: 436 [2560/8000 (32%)]	Loss: 30.540567
Train Epoch: 436 [3840/8000 (48%)]	Loss: 30.555437
Train Epoch: 436 [5120/8000 (64%)]	Loss: 30.558439
Train Epoch: 436 [6400/8000 (80%)]	Loss: 30.458492
Train Epoch: 436 [7680/8000 (96%)]	Loss: 30.269566
====> Epoch: 436 Average loss: 30.4524
Train Epoch: 437 [0/8000 (0%)]	Loss: 30.645180
Train Epoch: 437 [1280/8000 (16%)]	Loss: 30.267159
Train Epoch: 437 [2560/8000 (32%)]	Loss: 30.503212
Train Epoch: 437 [3840/8000 (48%)]	Loss: 30.374989
Train Epoch: 437 [5120/8000 (64%)]	Loss: 30.482668
Train Epoch: 437 [6400/8000 (80%)]	Loss: 30.743961
Train Epoch: 437 [7680/8000 (96%)]	Loss: 30.409904
====> Epoch: 437 Average loss: 30.4434
Train Epoch: 438 [0/8000 (0%)]	Loss: 30.232430
Train Epoch: 438 [1280/8000 (16%)]	Loss: 30.314926
Train Epoch: 438 [2560/8000 (32%)]	Loss: 30.534637
Train Epoch: 438 [3840/8000 (48%)]	Loss: 31.342625
Train Epoch: 438 [5120/8000 (64%)]	Loss: 30.179022
Train Epoch: 438 [6400/8000 (80%)]	Loss: 30.352499
Train Epoch:

Train Epoch: 457 [2560/8000 (32%)]	Loss: 30.659775
Train Epoch: 457 [3840/8000 (48%)]	Loss: 30.559347
Train Epoch: 457 [5120/8000 (64%)]	Loss: 30.397949
Train Epoch: 457 [6400/8000 (80%)]	Loss: 30.327604
Train Epoch: 457 [7680/8000 (96%)]	Loss: 30.381212
====> Epoch: 457 Average loss: 30.4253
Train Epoch: 458 [0/8000 (0%)]	Loss: 30.398132
Train Epoch: 458 [1280/8000 (16%)]	Loss: 30.276541
Train Epoch: 458 [2560/8000 (32%)]	Loss: 30.580500
Train Epoch: 458 [3840/8000 (48%)]	Loss: 30.328581
Train Epoch: 458 [5120/8000 (64%)]	Loss: 30.351604
Train Epoch: 458 [6400/8000 (80%)]	Loss: 30.423641
Train Epoch: 458 [7680/8000 (96%)]	Loss: 30.581821
====> Epoch: 458 Average loss: 30.4261
Train Epoch: 459 [0/8000 (0%)]	Loss: 30.531387
Train Epoch: 459 [1280/8000 (16%)]	Loss: 30.530066
Train Epoch: 459 [2560/8000 (32%)]	Loss: 30.424150
Train Epoch: 459 [3840/8000 (48%)]	Loss: 30.370459
Train Epoch: 459 [5120/8000 (64%)]	Loss: 30.699116
Train Epoch: 459 [6400/8000 (80%)]	Loss: 30.375307
Train Epoch:

Train Epoch: 478 [3840/8000 (48%)]	Loss: 30.588358
Train Epoch: 478 [5120/8000 (64%)]	Loss: 30.197462
Train Epoch: 478 [6400/8000 (80%)]	Loss: 30.327345
Train Epoch: 478 [7680/8000 (96%)]	Loss: 30.707314
====> Epoch: 478 Average loss: 30.4576
Train Epoch: 479 [0/8000 (0%)]	Loss: 30.487116
Train Epoch: 479 [1280/8000 (16%)]	Loss: 30.510454
Train Epoch: 479 [2560/8000 (32%)]	Loss: 30.315191
Train Epoch: 479 [3840/8000 (48%)]	Loss: 30.303473
Train Epoch: 479 [5120/8000 (64%)]	Loss: 30.319138
Train Epoch: 479 [6400/8000 (80%)]	Loss: 30.554728
Train Epoch: 479 [7680/8000 (96%)]	Loss: 30.247780
====> Epoch: 479 Average loss: 30.4540
Train Epoch: 480 [0/8000 (0%)]	Loss: 30.274591
Train Epoch: 480 [1280/8000 (16%)]	Loss: 30.523434
Train Epoch: 480 [2560/8000 (32%)]	Loss: 30.447350
Train Epoch: 480 [3840/8000 (48%)]	Loss: 30.845787
Train Epoch: 480 [5120/8000 (64%)]	Loss: 30.508322
Train Epoch: 480 [6400/8000 (80%)]	Loss: 30.362074
Train Epoch: 480 [7680/8000 (96%)]	Loss: 30.987005
====> Epoch:

Train Epoch: 499 [5120/8000 (64%)]	Loss: 30.576393
Train Epoch: 499 [6400/8000 (80%)]	Loss: 30.169477
Train Epoch: 499 [7680/8000 (96%)]	Loss: 30.436518
====> Epoch: 499 Average loss: 30.4428
Train Epoch: 500 [0/8000 (0%)]	Loss: 30.663279
Train Epoch: 500 [1280/8000 (16%)]	Loss: 30.400286
Train Epoch: 500 [2560/8000 (32%)]	Loss: 30.319023
Train Epoch: 500 [3840/8000 (48%)]	Loss: 30.092892
Train Epoch: 500 [5120/8000 (64%)]	Loss: 30.510504
Train Epoch: 500 [6400/8000 (80%)]	Loss: 30.523155
Train Epoch: 500 [7680/8000 (96%)]	Loss: 30.087650
====> Epoch: 500 Average loss: 30.4398
Train Epoch: 501 [0/8000 (0%)]	Loss: 30.266592
Train Epoch: 501 [1280/8000 (16%)]	Loss: 30.212223
Train Epoch: 501 [2560/8000 (32%)]	Loss: 30.642862
Train Epoch: 501 [3840/8000 (48%)]	Loss: 30.149555
Train Epoch: 501 [5120/8000 (64%)]	Loss: 30.425575
Train Epoch: 501 [6400/8000 (80%)]	Loss: 30.181929
Train Epoch: 501 [7680/8000 (96%)]	Loss: 30.607433
====> Epoch: 501 Average loss: 30.4474
Train Epoch: 502 [0/8000